# Extract data from .sol files

Use this if you are using the Wilkens function mode 2 to extract <a> loop (phase 0) and <c> loop (phase 1)densities

This workbook is an example of how the extractDataDir function can be used to extract physical paramaters from a folder a .sol files. 

This data frame is then manipulated for plotting data from a synchrotron experiment as an example.

## Import stuff

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mplcursors
%matplotlib qt5

from src.cmwp_tools import extractDataDir
from src.xrd_tools import addLabels

## Extract data from solution files

This code extracts data from CMWP .sol files

In [ ]:
directory = '/home/rhys/Documents/CMWP-211102/2020_11_DESY/hyd/'
data_df = extractDataDir(directory+'*.sol', calcLoop=False, wavelength=0.018505104)
unique = [20.695, 21.195, 21.695]
edges = [14.918, 14.918, 14.919]
dpa_val = 0.1
name='0.1_'

In [ ]:
directory = '/home/rhys/Documents/CMWP-211102/2020_11_DESY/hyd_3p/'
data_df = extractDataDir(directory+'*.sol', calcLoop=False, wavelength=0.018505104)
unique = [20.695, 21.195, 21.695]
edges = [14.918, 14.918, 14.919]
dpa_val = 0.1
name='0.1_3p'

In [ ]:
directory = '/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa2/'
data_df = extractDataDir(directory+'*.sol', calcLoop=False, wavelength=0.018505104)
unique = [9.245, 9.745, 10.245, 10.745, 11.245]
edges = [14.838, 14.836, 14.834, 14.832, 14.830]
dpa_val = 2
name='2'

In [ ]:
directory = '/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa05/'
data_df = extractDataDir(directory+'*.sol', calcLoop=False, wavelength=0.018505104)
#print(data_df['y'].unique())
unique = [-5.5, -5, -4.5, -4, -3.5]
edges = [14.780, 14.782, 14.782, 14.782, 14.782]
dpa_val = 0.5
name='0.5'

## Pull out z and y values from filename and calculate depth from edge

In [ ]:
data_df[['f2', 'y', 'f4', 'zs']] = data_df['filename'].str.split('_',expand=True)               # Split by '_'
data_df[['z', 'no']] = data_df['zs'].str.split('.s',expand=True)                                # Split by '.s'
data_df.drop(['f2', 'f4', 'zs', 'no'], axis=1, inplace=True)                        # Clear un-needed columns
data_df['y'] = pd.to_numeric(data_df['y'], downcast="float")
data_df['z'] = pd.to_numeric(data_df['z'], downcast="float")

# Rearrange to put y and z first
cols = data_df.columns.tolist()
data_df=data_df[cols[-2:] + cols[:-2]]


In [ ]:
data_df.insert(0, 'depth', data_df['y'])
data_df['depth'].replace(to_replace=unique, value=edges, inplace=True)
data_df['depth'] = data_df['z'] - data_df['depth']
data_df['depth'] = np.round(data_df['depth'] * 1000,1)

In [ ]:
data_df

## Calculate DPA

In [ ]:
depth, dpa = np.loadtxt('data/srim.txt')

dpa *= dpa_val

depth_int = list(depth)+list((depth[0:-1]+depth[1:])/2)
dpa_int = list(dpa)+list((dpa[0:-1]+dpa[1:])/2)

index = np.argsort(depth_int)
depth_int = np.array(depth_int)[index]
dpa_int = np.array(dpa_int)[index]

In [ ]:
data_df['dpa'] = [dpa_int[np.argmin(np.abs(i-depth_int))] for i in data_df['depth']]

## Get a list of phase names, hkls, positions

In [ ]:
phases = np.array(data_df['peak_phase'][20])
poss = np.array(data_df['peak_pos'][20])
names = np.array(data_df['peak_names'][20])

phaseNames = ['Zr', 'SPP', 'ZrH']
hkls = [names[phases==0], names[phases==2], names[phases==3]]
peakPos = [poss[phases==0], poss[phases==2], poss[phases==3]]
colours = ['b', 'r', 'g']

## Plot every other

In [ ]:
fnames = list(directory + data_df[(data_df['y'] == unique[2]) & (data_df['depth'] > 2)]['filename'].str[:-4]+'.dat')[::2][:9]
depths = data_df[(data_df['y'] == unique[2]) & (data_df['depth'] > 2)]['depth'][::2][:9]
dpas = data_df[(data_df['y'] == unique[2]) & (data_df['depth'] > 2)]['dpa'][::2][:9]

from matplotlib.cm import get_cmap
cmap = get_cmap('viridis')

colors = [cmap(i) for i in np.linspace(0, 0.9, len(fnames))]  # Linear colour against depth
colors = [cmap(i/np.max(dpas)) for i in dpas]                 # Linear colour against dpa

f, (lax, ax) = plt.subplots(2,1, gridspec_kw={'height_ratios': [1, 5]}, figsize = (14, 8), sharex=True)

i=0
for fname, depth, c, dpa in zip(fnames, depths, colors, dpas):
    x, y = np.loadtxt(fname, unpack=True)
    ax.plot(x,np.log(y)+i, 'o-', markersize=1,  color=c, lw=1, label=str(depth)[:-2] + 'um = ' + str(dpa)[:4] + ' dpa')
    i-=0.2
    
plt.legend()
#plt.yscale('log')
plt.xlim(np.min(x), np.max(x))


# plot labels

addLabels(ax=ax, lax=lax, names=phaseNames, hkls=hkls, positions=peakPos, colours=colours)

## Plot raw data

In [ ]:
def plotfig(num):
    
    f, (lax, ax) = plt.subplots(2,1, gridspec_kw={'height_ratios': [1, 5]}, figsize = (14, 8), sharex=True)
   
    data = list(directory + data_df['filename'].str[:-4]+'.dat')[num]
    back = list(directory + data_df['filename'].str[:-4]+'.int.bg.dat')[num]
    calc = list(directory + data_df['filename'].str[:-4]+'.int.th.dat')[num]

    # Plot data
    x_data, y_data = np.loadtxt(data, unpack=True)
    ax.plot(x_data, y_data, '--', lw=1, c='g', label='measured')
    
    # Plot calculated
    x, y = np.loadtxt(calc, unpack=True)
    ax.plot(x, y, lw=1, c='r', label='calculated')

    # Plot background
    x, y = np.loadtxt(back, unpack=True)
    ax.plot(x, y, '--', c='b', lw=1, label='background')

    plt.legend()
    plt.yscale('log')
    plt.xlim(np.min(x), np.max(x))
    
    plt.xlabel('2theta (degrees)')
    plt.ylabel('Intensity')

    
    # plot labels
    
    addLabels(ax=ax, lax=lax, names=phaseNames, hkls=hkls, positions=peakPos, colours=colours)
    
plotfig(40)

## Plot

This plots dislocation density, <a> loop fraction, M paramater and lattice paramaters as a function of depth from sample edge

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(16,12))
fig.subplots_adjust(left=0.06, right=0.96, top=0.98, bottom=0.1, hspace=0.3, wspace = 0.35)
axes = axes.ravel()

max_x = 40

# Plot the dpa
for sax in axes:
    sax.set_xlabel('Depth [${\mu}m$]', fontsize=12)
    sec = sax.twinx()
    sec.fill(depth, np.array(dpa), alpha=0.2)
    sec.tick_params(axis='both', which='major', labelsize=10)
    sec.set_xlim(0, max_x)
    sec.set_ylim(0)
    sec.set_ylabel('Dose [dpa]', fontsize=12)
    
    sax.set_zorder(sec.get_zorder()+1)
    sax.patch.set_visible(False);

# Split the lattice param axis
lim_defs=[np.round(np.min(data_df['a_lat'])-0.002,3), np.round(np.max(data_df['a_lat'])+0.002,3), 
          np.round(np.min(data_df['c_lat'])-0.002,3), np.round(np.max(data_df['c_lat'])+0.002,3)]

a_ratio = (lim_defs[1]-lim_defs[0])/((lim_defs[3]-lim_defs[2])+(lim_defs[1]-lim_defs[0]))

pos_orig = axes[6].get_position() # get the original position 
axes = np.append(axes, plt.axes([pos_orig.x0, pos_orig.y0 + (pos_orig.height * (1-a_ratio)),  pos_orig.width, pos_orig.height * a_ratio] ))
axes[9].xaxis.set_ticklabels([])
axes[6].set_position([pos_orig.x0, pos_orig.y0,  pos_orig.width, pos_orig.height * (1-a_ratio)])

# Common settings for all axes
for sax in axes:
    sax.set_xlim(0, max_x)
    sax.tick_params(axis='both', which='major', labelsize=10)
    sax.xaxis.set_ticks_position('both')

#### dislocation density

axes[0].errorbar(x=data_df['depth'], y=data_df['d'], yerr = [data_df['d_neg'],data_df['d_pos']],  marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[0].set_ylabel('<a> dislocation density [$10^{14} m^{-2}$]', fontsize=12)
axes[0].set_ylim(0)

#### m

d_lim = 0.5

axes[1].errorbar(x=data_df['depth'][data_df['d']>d_lim], y=np.exp(2)*data_df['e'][data_df['d']>d_lim], 
                 yerr = np.exp(2)*np.array([data_df['e_neg'][data_df['d']>d_lim],data_df['e_pos'][data_df['d']>d_lim]]), marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)

axes[1].set_ylabel('$M_{<a>}$', fontsize=12)
axes[1].set_ylim(0)

#### dislocation density

axes[3].errorbar(x=data_df['depth'], y=data_df['d_1'], yerr = [data_df['d_1_neg'],data_df['d_1_pos']],  marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[3].set_ylabel('<c> dislocation density [$10^{14} m^{-2}$]', fontsize=12)
axes[3].set_ylim(0)

#### m
axes[4].errorbar(x=data_df['depth'][data_df['d_1']>d_lim], y=np.exp(2)*data_df['e_1'][data_df['d_1']>d_lim], 
                 yerr = np.exp(2)*np.array([data_df['e_1_neg'][data_df['d_1']>d_lim],data_df['e_1_pos'][data_df['d_1']>d_lim]]), marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[4].set_ylabel('$M_{<c>}$', fontsize=12)
axes[4].set_ylim(0)

#### a loop fraction
data_df['a_frac'] = data_df['d']/(data_df['d']+data_df['d_1'])
data_df.plot(ax=axes[2], x='depth', y='a_frac', ms=3, marker='o', c='k', lw=0,  legend=False, xlabel='Depth [${\mu}m$]', ylabel='$f_{<a>}$', ylim=(0,1))

#### c loop fraction
data_df['c_frac'] = data_df['d_1']/(data_df['d']+data_df['d_1'])
data_df.plot(ax=axes[5], x='depth', y='c_frac', ms=3, marker='o', c='k', lw=0,  legend=False, xlabel='Depth [${\mu}m$]', ylabel='$f_{<c>}$', ylim=(0,1))

#### c paramater
data_df.plot(ax=axes[6], x='depth', y='c_lat', yerr='c_err', ms=3, marker='o', c='k', lw=0, capsize=5, elinewidth=1, legend=False, xlabel='Depth [${\mu}m$]', ylabel='c [Å]', ylim=(lim_defs[2], lim_defs[3]))
axes[6].set_yticks(np.arange(lim_defs[2], lim_defs[3], 0.001)[:-1])

#### a paramater
data_df.plot(ax=axes[9], x='depth', y='a_lat', yerr='a_err', ms=3, marker='o', c='k', lw=0, capsize=5, elinewidth=1, legend=False, xlabel='', ylabel='a [Å]', ylim=(lim_defs[0], lim_defs[1]))
axes[9].set_yticks(np.arange(lim_defs[0], lim_defs[1], 0.001))

#### c/a
data_df.plot(ax=axes[7], x='depth', y='covera', yerr='covera_err', ms=3, marker='o', c='k', lw=0, capsize=5, elinewidth=1, legend=False, xlabel='Depth [${\mu}m$]', ylabel='c/a ratio')

#### cell volume
data_df.plot(ax=axes[8], x='depth', y='covera', yerr='covera_err', ms=3, marker='o', c='k', lw=0, capsize=5, elinewidth=1, legend=False, xlabel='Depth [${\mu}m$]', ylabel='Cell volume [$Å^3$]')

plt.savefig('/home/rhys/Dropbox (Research Group)/Postdoc/DESY/Results_new/'+name+'_ellip1.pdf', dpi=600)
plt.show()

for ax in axes:
    crs = mplcursors.cursor(ax,hover=False)

    crs.connect("add", lambda sel: sel.annotation.set_text('y={:.3f}, z={:.3f}'.format(data_df['y'][int(sel.index)], data_df['z'][int(sel.index)])))
    crs.connect("add", lambda sel: plotfig(int(sel.index)))
    plt.show()
    

In [ ]:
plt.plot(data_df['dpa'][data_df['depth']>2], data_df['d'][data_df['depth']>2], ms=3, marker='o', c='k', lw=0)

In [ ]:
depth, dpa = np.loadtxt('data/srim.txt')

dpa *= dpa_val

plt.plot(depth,dpa)